In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [2]:
import os

In [3]:
# Load models

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    max_completion_tokens=200
)

In [4]:
# Load PDF
pdf_link = 'Lei_geral_protecao_dados_pessoais_1ed.pdf'

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()  # Carregar o pdf e separar em páginas


In [5]:
# Separar em Chunks (Pedaços de documento)

text__splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,  # Número de palavras que se sobrepõem entre os chunks
    length_function=len,
    add_start_index=True
)

chunks = text__splitter.split_documents(pages)  # Separar o documento em pedaços

In [6]:
# Salvar no vector DB
db = Chroma.from_documents(chunks,embeddings_model, persist_directory="text_index")

In [8]:
# carregar DB
vectorDB = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

# Load Retriever (buscador)
retriever = vectorDB.as_retriever(search_kwargs={"k": 3})  # k = número de documentos retornados

# Load QA Chain (construção da cadeia do chunks para fazer o llm)
qa_chain = load_qa_chain(
    llm,
    chain_type="stuff", # "stuff" é o tipo de cadeia que queremos usar
    verbose=True   # Mostra o que está a acontecer
)

In [9]:
def ask(question):
    context = retriever.get_relevant_documents(question)  # Pega os documentos relevantes
    answer = (qa_chain({"input_documents": context, "question": question},return_only_outputs=True))['output_text']  # Faz a pergunta
    return answer

In [10]:
user_question = input("user: ")
answer = ask(user_question) # Faz a pergunta
print(answer)  # Imprime a resposta

/tmp/ipykernel_943170/738414472.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(question)  # Pega os documentos relevantes
/tmp/ipykernel_943170/738414472.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = (qa_chain({"input_documents": context, "question": question},return_only_outputs=True))['output_text']  # Faz a pergunta




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Lei Geral de Proteção  
de Dados Pessoais

Lei Geral de Proteção  
de Dados Pessoais

Lei Geral de Proteção  
de Dados Pessoais
Human: Quais as principais leis que preciso para aplicar em minha empresa?

> Finished chain.

> Finished chain.
Uma das principais leis que você precisa aplicar em sua empresa é a Lei Geral de Proteção de Dados Pessoais (LGPD). Ela estabelece regras sobre a coleta, armazenamento, tratamento e compartilhamento de dados pessoais, visando proteger a privacidade e a segurança das informações dos usuários. Além da LGPD, outras leis importantes a serem consideradas dependem do ramo de atuação da sua empresa, como o Código de Defesa do Consumidor, a CLT (Consolidação das Leis do T